In [1]:
from dwave.system.samplers import DWaveSampler
from minorminer import find_embedding
from dwave.system.composites import FixedEmbeddingComposite
import numpy as np
from scipy.sparse import dok_matrix

In [2]:
solver = "DW_2000Q_6"
sampler = DWaveSampler(solver=solver)

qubo = [[-0.25,     1],
        [    0, -0.25]] #,
        #[    0,     0, -0.25]]
dok_qubo = dok_matrix(qubo)
qubo_dict = {k: dok_qubo[k] for k in dok_qubo.keys()}

offset0 = 0 #-0.05
offset1 = -0.05
tries = 100

In [3]:
qpu_graph = sampler.edgelist
anneal_offset = np.zeros(2048)
for _ in range(tries):
    embedding = find_embedding(qubo_dict, qpu_graph)
    embedding_idx = [idx for embed_list in embedding.values() for idx in embed_list]
    embed = FixedEmbeddingComposite(sampler, embedding)
    anneal_offset_ranges = np.array(
        embed.properties["child_properties"]["anneal_offset_ranges"]
    )
    offset_range = anneal_offset_ranges[embedding_idx]
    range0 = offset_range[0][0] <= offset0 <= offset_range[0][1]
    range1 = offset_range[1][0] <= offset1 <= offset_range[1][1]
    if range0 and range1:
        anneal_offset[embedding_idx[0]] = offset0
        anneal_offset[embedding_idx[1]] = offset1
        break
print(anneal_offset_ranges[embedding_idx])
print(embedding)
print(qubo_dict)

[[-0.13342708  0.10839509]
 [-0.1388919   0.07703513]]
{0: [1280], 1: [1408]}
{(0, 0): -0.25, (0, 1): 1.0, (1, 1): -0.25}


In [4]:
dwave_config = {
    "annealing_time": 1,  # integer microseconds [1, 2000]
    "answer_mode": "histogram",  # histogram or raw
    "auto_scale": False,
    "num_reads": 1000,  # raw will dump out all results
    "num_spin_reversal_transforms": 0,
    "readout_thermalization": 0,  # default 0
    "programming_thermalization": 1000,  # default 1000
    "chain_strength": 1,
    "anneal_offsets": list(anneal_offset),
}

In [5]:
result = embed.sample_qubo(qubo_dict, **dwave_config)
raw = result.to_pandas_dataframe()

In [6]:
raw

,0,1,chain_break_fraction,energy,num_occurrences
0,0,1,0.0,-0.25,435
1,1,0,0.0,-0.25,536
2,0,0,0.0,0.00,29


In [7]:
#basis = [(i, j, k) for i in [0, 1] for j in [0, 1] for k in [0, 1]]
basis = [(i, j) for i in [0, 1] for j in [0, 1]]

for I in range(2 ** 2):
    dwstate = np.array([int(i) for i in '{0:02b}'.format(I)])
    print(I, basis[I], dwstate)

0 (0, 0) [0 0]
1 (0, 1) [0 1]
2 (1, 0) [1 0]
3 (1, 1) [1 1]


In [8]:
# Check result

In [9]:
# evaluate the QUBO given binary vector b and return energy F
def eval_QUBO(Q, b):
    #F = np.dot(b, np.dot(Q, b) )
    F = np.einsum('i,ij,j',b,Q,b)
    return F

# Convert non-negative n-bit integer to n-bit binary representation and return numpy array
def int_to_bin(a, n):
    l = int(a).bit_length(); # length of binary conversion
    # Check that the bit length fits the b-bit representation
    if l > n:
        print(" <<Bit length exceeds repreesntation size>>")
        raise ValueError
    x = bin(int(a)); # binary converstion returns string x
    y =x[2:l+2] # store last l chars of x in y
    b = np.zeros(n);
    for i in range(len(y)):
    	b[n-l+i] = int(y[i]); # add the bits from smallest to largest in the last l slots
    return b

# Find argument that minimizes QUBO F(b) = b^T Q b and return as numpy array
def argmin_QUBO(Q):
    n = len(Q);
    bx = int_to_bin(0, n);
    Fx = eval_QUBO(Q, bx);
    result = []
    for a in range(2**n): # loop over all 2^n possibilities
        b = int_to_bin(a, n); 
        F = eval_QUBO(Q, b);
        #print('%s  %s' %(b,F))
        result.append([F,b])
        if F < Fx:
            Fx = F
            bx = b
    return bx,result

In [10]:
bx, result = argmin_QUBO(qubo)
result.sort(key=lambda x: x[0])

In [11]:
result

[[-0.25, array([0., 1.])],
 [-0.25, array([1., 0.])],
 [0.0, array([0., 0.])],
 [0.5, array([1., 1.])]]